In [74]:
# Imports
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
from IPython.display import Markdown, display
import os
import json
import requests
import gradio as gr

In [75]:
# load dotenv file and initialize openai client
load_dotenv(override=True)
openai_client = OpenAI()

In [76]:
# Set variables for using Pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"
if pushover_user:
    print(f"PushOver user exists and starts with {pushover_user[:5]}")
else:
    print("Pushover user doesn't exists. Please add to send push notifications.")
if pushover_token:
    print(f"PushOver token exists and starts with {pushover_token[:5]}")
else:
    print("Pushover token doesn't exists. Please add to send push notifications.")

PushOver user exists and starts with u8a37
PushOver token exists and starts with asdoy


In [77]:
# Def push method to send push messages to the user
def push(message):
    print(f"Push:{message}")
    payload = {"user":pushover_user, "token":pushover_token, "message":message}
    requests.post(pushover_url,data=payload)

In [78]:
# Quickly test push message
#push("Hello! Can you see this?")

In [79]:
# Create a record_user_details function that will record the user details who looks interested in your profile
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email: {email} and notes: {notes}")
    return {"Recorded":"OK"}


In [80]:
# Create a record_unknown_question function that will record the Questions that AI agent can't answer
def record_unknown_question(question):
    push(f"Recording {question} that I couldn't answer.")
    return {"Recorded":"Ok"}

In [81]:
# Create a json file for record_user_details function to give a context to LLM about the function which is going to be used as a tool by LLM
record_user_details_json={
    "name":"record_user_details",
    "description": "Use this tool to records the details of user if they are interested in being in touch and provided their email id",
    "parameters":{
        "type":"object",
        "properties":{
            "email":{
                "type": "string",
                "description": "The email id of this user"
                    },
            "name":{
                "type": "string",
                "description": "The name of this user if they provided it"
                    },
            "notes":{
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
                    }},
        "required": ["email"],
        "additionalProperties": False
        }
}

In [82]:
# Create a json file for record_unknown_question function to share a context with LLM about this function that is going to provided as a Tool
record_unknown_question_json ={
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that you can't answer as you don't know the answer, even if it's about something trivial or unrelated to career.",
    "parameters":{
        "type":"object",
        "properties":{
            "question":{
                "type": "string",
                "description": "The question asked that can't be answered."}
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [83]:
# Create a list of tools for the LLM
tools = [{"type":"function", "function":record_user_details_json},
        {"type":"function", "function":record_unknown_question_json}]

In [84]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to records the details of user if they are interested in being in touch and provided their email id',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email id of this user'},
     'name': {'type': 'string',
      'description': 'The name of this user if they provided it'},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that you can't answer as you don't know the answer, even if it's about something trivial or unrelated to career.",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'descript

In [85]:
# Create a handle_tool_calls function to handle the calls to the tools
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool Called: {tool_name} \nArguments: {arguments}", flush=True)
        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
        print(f"result:{result}")
        # We can also use global() function to call tool function as it is based on the name, if the name is a then call function a as below
        # tool = globals.get(tool_name)
        # result = tool(**arguments) if tool else {}

        results.append({"role":"tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results

In [86]:
# Generate resources by providing linkedin profil and quick summary about yourself
reader = PdfReader("me\SumanPrakash_LinkedIn.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin+=text
with open("me\summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
name = "Suman Prakash"

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\suman\AppData\Local\Temp\ipykernel_25708\2763357742.py:2: SyntaxWarning: invalid escape sequence '\S'
  reader = PdfReader("me\SumanPrakash_LinkedIn.pdf")
C:\Users\suman\AppData\Local\Temp\ipykernel_25708\2763357742.py:8: SyntaxWarning: invalid escape sequence '\s'
  with open("me\summary.txt", "r", encoding="utf-8") as f:


In [87]:
# Generate a system prompt 
system_prompt = f"""You are acting as {name}. You are answering questions on {name}'s website, 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. 
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
If you don't know the answer, say so.

With this context, please chat with the user, always staying in character as {name}.

PFB the summary and Linkedin Profile as resource that you need to refer for answering questions or 
communicating with the user on other side.

## Summary:\n{summary}\n\n
## LinkedIn Profile:\n{linkedin}

NOTE: You can use following tools provided to you:
1. If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool.
2. If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career.
"""

In [88]:
print(system_prompt)

You are acting as Suman Prakash. You are answering questions on Suman Prakash's website, 
particularly questions related to Suman Prakash's career, background, skills and experience. 
Your responsibility is to represent Suman Prakash for interactions on the website as faithfully as possible. 
You are given a summary of Suman Prakash's background and LinkedIn profile which you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
If you don't know the answer, say so.

With this context, please chat with the user, always staying in character as Suman Prakash.

PFB the summary and Linkedin Profile as resource that you need to refer for answering questions or 
communicating with the user on other side.

## Summary:
My Name is Suman Prakash. I am a Data Analyst, QA, Writer and entrepreneur. I developed a keen interest in Artificial intelligence.
And, exploring the field. 
My native place is Patna, Bih

In [89]:
# Create a chat function where LLM chats with the user on your behalf and use necessary tools when required
def chat(message, history):
    messages = [{"role":"system", "content":system_prompt}] + history + [{"role":"user", "content":message}]
    done = False
    while not done:
        response = openai_client.chat.completions.create(model="gpt-4o-mini", messages= messages, tools=tools)
        finish_reason = response.choices[0].finish_reason

        if finish_reason=="tool_calls":
            print(response.choices[0])
            message=response.choices[0].message
            tool_calls= message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content



In [ ]:
gr.ChatInterface(chat).launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_pYMwaKzIx3TOJseVmvhOgv9A', function=Function(arguments='{"question":"Who is your favorite singer?"}', name='record_unknown_question'), type='function')]))
Tool Called: record_unknown_question 
Arguments: {'question': 'Who is your favorite singer?'}
Push:Recording Who is your favorite singer? that I couldn't answer.
result:{'Recorded': 'Ok'}
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_FG2K9hrdBPzpsqqASnbqhx5X', function=Function(arguments='{"email":"john_doe@abc.com","name":"John Doe","notes":"CEO looking to hire talented individuals."}', name='record_use